1. Carga de datos:
– Descarga y carga el dataset «Titanic» desde Kaggle. Realiza una exploración inicial de las variables disponibles, que incluyen información sobre la edad, el género, la clase del pasajero, entre otros.
– Examina cuántos valores faltan en las variables importantes como la edad y el precio del boleto.


In [1]:
import pandas as pd
df = pd.read_csv('../Data/train.csv')
df.head().T

,0,1,2,3,4
PassengerId,1,2,3,4,5
Survived,0,1,1,1,0
Pclass,3,1,3,1,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry"
Sex,male,female,female,female,male
Age,22.0,38.0,26.0,35.0,35.0
SibSp,1,1,0,1,0
Parch,0,0,0,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450
Fare,7.25,71.2833,7.925,53.1,8.05


In [2]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
print(df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [4]:
df['FamilySize'] = df['SibSp'] + df['Parch'] +1 #El +1 es para incluir al pasajero
df['CabinKnow'] = df['Cabin'].notna().astype(int) #Si se conoce la cabina 1 y si no 0.
df['FareCategory'] = df['Fare'].apply(lambda x: 'Low' if x <= 7.91 else 'Medium' if x <= 14.454 else 'High')
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())


In [5]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

#Selecciono la variable objetivo y la separo
X = df.drop(['Survived'], axis = 1)
y = df['Survived'] #Esta sería mi columna objetivo

num_columns = X.select_dtypes(include=['float64', 'int64']).columns
cat_columns = X.select_dtypes(include=['object', 'category']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps =[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore') )
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_columns),
        ('cat', categorical_transformer, cat_columns)
    ]
)

#Ahora hago el pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])


In [6]:
print(df.isnull().sum())

PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age             177
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin           687
Embarked          2
FamilySize        0
CabinKnow         0
FareCategory      0
Title             0
dtype: int64


4. Evaluación de nuevas características:
– Aplica un modelo de machine learning (como un modelo de regresión logística o un árbol de decisión) antes y después de agregar las nuevas características para evaluar su impacto en el rendimiento del modelo.
– Utiliza métricas como la exactitud y el F1-score para comparar el rendimiento con y sin las nuevas variables.

MODELO CON VARIABLES CREADAS

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Creo el modelo
pipeline_LR = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('classifier', LogisticRegression())
])

#Entreno el modelo
pipeline_LR.fit(X_train, y_train)

#Hago las predicciones
y_pred_LR = pipeline_LR.predict(X_test)

#Evaluo el modelo
print(f'Reporte del modelo Logistic Regression:')
print(classification_report(y_test, y_pred_LR))

#Veo su accuracy 
accuracy_LR = accuracy_score(y_test, y_pred_LR)

print(f'Accuracy del modelo Logistic Regression: {accuracy_LR}')

Reporte del modelo Logistic Regression:
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       105
           1       0.78      0.78      0.78        74

    accuracy                           0.82       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.82      0.82      0.82       179

Accuracy del modelo Logistic Regression: 0.8212290502793296


MODELO SIN LAS VARIABLES NUEVAS

In [8]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

#Con esto vuelvo a cargar el dataset para que esté sin las variables nuevas
df = pd.read_csv('../Data/train.csv')

#Selecciono la variable objetivo y la separo
X = df.drop(['Survived'], axis = 1)
y = df['Survived'] #Esta sería mi columna objetivo

num_columns = X.select_dtypes(include=['float64', 'int64']).columns
cat_columns = X.select_dtypes(include=['object', 'category']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps =[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore') )
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_columns),
        ('cat', categorical_transformer, cat_columns)
    ]
)

#Ahora hago el pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

df.head().T

,0,1,2,3,4
PassengerId,1,2,3,4,5
Survived,0,1,1,1,0
Pclass,3,1,3,1,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry"
Sex,male,female,female,female,male
Age,22.0,38.0,26.0,35.0,35.0
SibSp,1,1,0,1,0
Parch,0,0,0,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450
Fare,7.25,71.2833,7.925,53.1,8.05


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Creo el modelo
pipeline_LR = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

#Entreno el modelo
pipeline_LR.fit(X_train, y_train)

#Hago las predicciones
y_pred_LR = pipeline_LR.predict(X_test)

#Evaluo el modelo
print(f'Reporte del modelo Logistic Regression:')
print(classification_report(y_test, y_pred_LR))

#Veo su accuracy 
accuracy_LR = accuracy_score(y_test, y_pred_LR)

print(f'Accuracy del modelo Logistic Regression: {accuracy_LR}')

Reporte del modelo Logistic Regression:
              precision    recall  f1-score   support

           0       0.83      0.88      0.85       105
           1       0.81      0.74      0.77        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179

Accuracy del modelo Logistic Regression: 0.8212290502793296


- Las nuevas variables han demostrado ser útiles para mejorar el recall de los sobrevivientes, lo que implica que el modelo con nuevas características puede identificar correctamente a más sobrevivientes, aunque a costa de una ligera disminución en la precision.
- El objetivo de mejorar el rendimiento del modelo mediante la creación de nuevas variables ha sido parcialmente alcanzado, ya que el modelo con nuevas variables muestra un mejor balance entre precision y recall gracias al F1-score más alto, aunque con un pequeño trade-off en precision.
- Si bien el modelo sin nuevas variables tiene una mejor precision para los sobrevivientes, las nuevas variables han proporcionado una mejora en el recall, lo que podría ser más beneficioso en escenarios donde se desea identificar a la mayor cantidad de sobrevivientes posible.